In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
zip_path = '/content/gdrive/MyDrive/ML Dataset/FOG.zip'
import zipfile

# Specify the directory where you want to extract the files
extract_path = '/content/'

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the files to the specified directory
    zip_ref.extractall(extract_path)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, GRU, Dense
from tensorflow.keras.optimizers import Adam
from shutil import copyfile

file_path = '/content/freezing-of-gait-exploration-main/freezing-of-gait-exploration-main/sliding_window_dataset'
file_names = [
    'S01R01experiment0', 'S01R02experiment0',
    'S02R01experiment0', 'S02R02experiment0',
    'S03R01experiment0', 'S03R01experiment1', 'S03R02experiment0', 'S03R03experiment0',
    'S04R01experiment0',
    'S05R01experiment0', 'S05R01experiment1', 'S05R01experiment2', 'S05R01experiment3',
    'S06R01experiment0',
]
file_name = [
    'S07R01experiment0', 'S07R02experiment0',
    'S08R01experiment0', 'S08R01experiment1', 'S08R01experiment2', 'S08R01experiment3',
    'S09R01experiment0', 'S09R01experiment1'
]



# Define your model creation function
def create_model():
    # Define your model architecture
    model = Sequential()
    model.add(InputLayer((1000, 4)))
    model.add(LSTM(64))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

def train_model(X_train, y_train, X_test, y_test, initial_model=None):
    if initial_model is None:
          model = create_model()
    else:
          model = initial_model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=256)

    return model

# Load and preprocess your data
initial_model = None
for name in file_names:
    X = np.load(f'{file_path}/{name}_X.npy')
    y = np.load(f'{file_path}/{name}_y.npy')
    y = np.array([[i] for i in y])
    length = X.shape[0]
    test_data_idx_start = int(length * 0.8)
    X_train = X[:test_data_idx_start]
    y_train = y[:test_data_idx_start]
    X_test = X[test_data_idx_start:]
    y_test = y[test_data_idx_start:]
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    initial_model = train_model(X_train, y_train, X_test, y_test, initial_model)

# Save the final trained model
initial_model.save('/content/lstm_model.h5')


Streaming output truncated to the last 5000 lines.
87/87 [==============================] - 4s 47ms/step - loss: 0.0158 - accuracy: 0.9946 - val_loss: 0.0268 - val_accuracy: 0.9998
Epoch 108/200
87/87 [==============================] - 4s 47ms/step - loss: 0.0156 - accuracy: 0.9948 - val_loss: 0.0363 - val_accuracy: 0.9998
Epoch 109/200
87/87 [==============================] - 4s 47ms/step - loss: 0.0148 - accuracy: 0.9957 - val_loss: 0.0270 - val_accuracy: 0.9998
Epoch 110/200
87/87 [==============================] - 4s 47ms/step - loss: 0.0165 - accuracy: 0.9945 - val_loss: 0.0298 - val_accuracy: 0.9966
Epoch 111/200
87/87 [==============================] - 4s 47ms/step - loss: 0.0165 - accuracy: 0.9944 - val_loss: 0.0412 - val_accuracy: 0.9998
Epoch 112/200
87/87 [==============================] - 4s 47ms/step - loss: 0.0153 - accuracy: 0.9948 - val_loss: 0.0338 - val_accuracy: 0.9980
Epoch 113/200
87/87 [==============================] - 4s 47ms/step - loss: 0.0143 - accuracy: 0.99

In [6]:
for name in file_name:
    X1 = np.load(f'{file_path}/{name}_X.npy')
    length = X1.shape[0]
    y1 = np.load(f'{file_path}/{name}_y.npy')
    y1 = np.array([[i] for i in y1])
    test_data_idx_start = int(length)
    X1_test = X1[:test_data_idx_start]
    y1_test = y1[:test_data_idx_start]

In [8]:
from tensorflow.keras.models import load_model

model = load_model('/content/lstm_model.h5')
yhat = model.predict(X1_test)# metrics# metrics
from sklearn import metrics
TN, FP, FN, TP = metrics.confusion_matrix(y1_test, np.array([0 if y < .5 else 1 for y in yhat])).ravel()


print('Sensitivity, hit rate, recall, or true positive rate: ')
TPR = TP/(TP+FN)
print(TPR)
print('Specificity or true negative rate')
TNR = TN/(TN+FP)
print(TNR)
print('Precision or positive predictive value')
PPV = TP/(TP+FP)
print(PPV)
print('Negative predictive value')
NPV = TN/(TN+FN)
print(NPV)
print('Fall out or false positive rate')
FPR = FP/(FP+TN)
print(FPR)
print('False negative rate')
FNR = FN/(TP+FN)
print(FNR)
print('False discovery rate')
FDR = FP/(TP+FP)
print(FDR)

print('Overall accuracy')
ACC = (TP+TN)/(TP+FP+FN+TN)
print(ACC)

578/578 [==============================] - 7s 11ms/step
Sensitivity, hit rate, recall, or true positive rate: 
0.9919028340080972
Specificity or true negative rate
0.9648449039881831
Precision or positive predictive value
0.9114583333333334
Negative predictive value
0.996947496947497
Fall out or false positive rate
0.035155096011816836
False negative rate
0.008097165991902834
False discovery rate
0.08854166666666667
Overall accuracy
0.9720779220779221


In [9]:
from google.colab import files

files.download('/content/lstm_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>